In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import boxcox
pd.options.display.max_rows = 100
## Install xlrd package to load Excel files
#!conda install openpyxl
#!conda install xlrd

In [ ]:
file1 = pd.read_csv('file1.csv')
file2 = pd.read_csv('file2.csv')#, sep = '\t')
file3 = pd.read_csv('file3.csv')

In [ ]:
file1.head()

In [ ]:
file2.head()

In [ ]:
file3.head()

In [ ]:
#renaming the columns
file1.rename(columns={'ST':'State'}, inplace=True) # changed the column name ST into State
file2.rename(columns={'ST':'State'}, inplace=True) # changed the column name ST into State
file2.rename(columns={'GENDER':'Gender'}, inplace=True) # changed the column name GENDER into Gender
file1.rename(columns={'GENDER':'Gender'}, inplace=True) # changed the column name GENDER into Gender


In [ ]:
file1.head()

In [ ]:
file2.head()

In [ ]:
file3.head()

In [ ]:
# Arranging all the columns in file 2 amd file 3 in the order provided in file 1
file2 = file2[['Customer', 'State', 'Gender', 'Education', 'Customer Lifetime Value', 'Income', 'Monthly Premium Auto', 'Number of Open Complaints', 'Policy Type', 'Vehicle Class', 'Total Claim Amount']]
file3 = file3[['Customer', 'State', 'Gender', 'Education', 'Customer Lifetime Value', 'Income', 'Monthly Premium Auto', 'Number of Open Complaints', 'Policy Type', 'Vehicle Class', 'Total Claim Amount']]



In [ ]:
# concatenate the files file1, file2, file3
auto_data_file = pd.concat([file1,file2,file3], axis=0)
(auto_data_file)

In [ ]:
# changing the column names to lower
auto_data_file.columns=[i.lower() for i in auto_data_file.columns]
auto_data_file
auto_data_file.info()

In [ ]:
# we have to reset the index
auto_data_file.reset_index() # after that we have two indices


# drop some columns which are not so important
auto_data_file.drop(['customer'], axis=1, inplace = True)
#auto_data_file.drop(['state'], axis=1, inplace = True)
auto_data_file.head()

In [ ]:
# Removing duplicates

auto_data_file['gender'].unique()
def clean_gender(x):
    if x in ['M', 'Male']:
        return 'M'
    elif x in ['F', 'Femal', 'female']:
        return 'F'
    else:
        return 'NaN'
    
    
    
               

In [ ]:
auto_data_file['gender'] = list(map(clean_gender, auto_data_file['gender'])) 

In [ ]:
auto_data_file['gender'].unique()

In [ ]:
# removing unwanted characters from some columns
auto_data_file['customer lifetime value'] = auto_data_file['customer lifetime value'].astype('string') # changing column the entries in the column to string
auto_data_file['customer lifetime value'] = auto_data_file['customer lifetime value'].str.replace('%', '')
auto_data_file['customer lifetime value'] = auto_data_file['customer lifetime value'].astype('float') # changing column the entries in the column to string
auto_data_file.info()
auto_data_file.head()

# truncate/round-up the entries in the column 'customer lifetime value' to integer

auto_data_file['customer lifetime value'] = auto_data_file['customer lifetime value'].apply(np.floor)

auto_data_file.head()    

In [ ]:
# cleaning up the  'number of open complaints' column
auto_data_file['number of open complaints'] = auto_data_file['number of open complaints'].astype('string') # changing column the entries in the column to string

auto_data_file['number of open complaints'].fillna('missing', inplace=True)
#auto_data_file['number of open complaints'].unique()  
    

In [ ]:
# cleaning up 'number of open complaints' column: continued

auto_data_file['number of open complaints'] = auto_data_file['number of open complaints'].astype(str).str.replace('(.*)/0/(.*)','0', regex=True)
auto_data_file['number of open complaints'] = auto_data_file['number of open complaints'].astype(str).str.replace('(.*)/2/(.*)','2', regex=True)
auto_data_file['number of open complaints'] = auto_data_file['number of open complaints'].astype(str).str.replace('(.*)/1/(.*)','1', regex=True)
auto_data_file['number of open complaints'] = auto_data_file['number of open complaints'].astype(str).str.replace('(.*)/3/(.*)','3', regex=True)
auto_data_file['number of open complaints'] = auto_data_file['number of open complaints'].astype(str).str.replace('(.*)/5/(.*)','5', regex=True)
auto_data_file['number of open complaints'] = auto_data_file['number of open complaints'].astype(str).str.replace('(.*)/4/(.*)','4', regex=True)

auto_data_file['number of open complaints'] = auto_data_file['number of open complaints'].str.replace('missing','NaN', regex=True)
  

auto_data_file['number of open complaints'] = auto_data_file['number of open complaints'].astype('float')
    
auto_data_file['number of open complaints'].unique()  

# saving the Monday's work to a new csv

auto_data_file.to_csv('Monday.csv', index=False)

In [ ]:
auto_data_file['number of open complaints'].unique()
auto_data_file.head(100)

In [ ]:
## Tuesday's work starts here

# calling the csv from Monday
auto_data_file = pd.read_csv('Monday.csv')
auto_data_file.head()

# Replacing zeros in the income column with NaN

auto_data_file['income'].replace('0', 'NaN', inplace=True) 

# Replacing null values with the mean

def replace_null_value(dataframe, column):
    if dataframe[column].dtype != object:
        mean = dataframe[column].mean()
        dataframe[column].fillna(value=mean, inplace = True)
    else:
        pass

for clmn in auto_data_file.columns:
    replace_null_value(auto_data_file, clmn)
 


auto_data_file.columns

In [ ]:
# Bucketing the data
auto_data_file.rename(columns={'state':'zone'}, inplace=True) # changed the column name state into zone
def old_to_new(old, new, columnname):
    auto_data_file[columnname].replace(old, new, inplace = True)

old_to_new(['Washington', 'WA'], 'East', 'zone') 
old_to_new(['California', 'Cali'], 'West', 'zone')
old_to_new('Oregon', 'North West', 'zone')
old_to_new(['Arizona', 'Nevada', 'AZ'], 'Central', 'zone')


In [ ]:
auto_data_file.head(100)

In [ ]:
# Optional task on vehicle class: replace Luxury SUV and Luxury Car into Luxury Vehicle
old_to_new('Luxury SUV', 'Luxury Vehicle', 'vehicle class')
old_to_new('Luxury Car', 'Luxury Vehicle', 'vehicle class')
auto_data_file['vehicle class'].unique()

In [ ]:
auto_data_file['income'].max()

In [ ]:
# Removing the outliers using the IQR method

def remove_outlier(column_name, df):
    percentile25 = df[column_name].quantile(0.25)
    percentile75 = df[column_name].quantile(0.75)
    iqr = percentile75 - percentile25
    upper_limit = percentile75 + 1.5 * iqr
    lower_limit = percentile25 - 1.5 * iqr
    #if i in df[column_name] > upper_limit and df[column_name] < lower_limit:
    #    df.replace(i, value = "Outlier", inplace = True)
    df.replace(to_replace = df[column_name][df[column_name] > upper_limit], value = "Outlier", inplace = True)
    df.replace(to_replace = df[column_name][df[column_name] < lower_limit], value = "Outlier", inplace = True)
    number_of_values_replaced = f"you replaced {df[column_name][df[column_name] > upper_limit].shape[0] + df[column_name][df[column_name] < lower_limit].shape[0]} outlier values with the string Outlier"
    return number_of_values_replaced
  
remove_outlier('income', auto_data_file) 
remove_outlier('customer lifetime value', auto_data_file) 
remove_outlier('monthly premium auto', auto_data_file) 

In [ ]:
auto_data_file['monthly premium auto'].info()

In [ ]:
# saving the Tuesday's work to a new csv

auto_data_file.to_csv('Tuesday.csv', index=False)